In [36]:
# =============================================================================
# IMPORTS
# =============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports loaded")

✓ Imports loaded


In [37]:
# =============================================================================
# CONFIGURATION - EDIT THIS
# =============================================================================

# SECTOR ETF TO TRACK
SECTOR_ETF = 'XBI'  # Small cap biotech

# BENCHMARK
BENCHMARK = 'SPY'

# TIME PERIOD
START_DATE = '2024-01-01'
END_DATE = '2026-01-08'

# OUTPERFORMANCE THRESHOLDS TO TEST
OUTPERFORM_THRESHOLDS = [0.01, 0.015, 0.02, 0.025]  # 1%, 1.5%, 2%, 2.5%

# CONSECUTIVE DAYS TO CONFIRM ROTATION
CONSECUTIVE_DAYS = [2, 3, 4, 5]

# FORWARD RETURNS TO MEASURE
FORWARD_PERIODS = [5, 10, 15, 20]  # days after rotation confirmed

print("✓ Configuration set")
print(f"  Testing: {SECTOR_ETF} vs {BENCHMARK}")
print(f"  Date range: {START_DATE} to {END_DATE}")
print(f"  Thresholds: {[f'{t*100:.1f}%' for t in OUTPERFORM_THRESHOLDS]}")
print(f"  Consecutive days: {CONSECUTIVE_DAYS}")

✓ Configuration set
  Testing: XBI vs SPY
  Date range: 2024-01-01 to 2026-01-08
  Thresholds: ['1.0%', '1.5%', '2.0%', '2.5%']
  Consecutive days: [2, 3, 4, 5]


In [38]:
# =============================================================================
# RELATIVE STRENGTH CALCULATION
# =============================================================================

def calculate_relative_strength(sector_df, benchmark_df):
    """
    Calculate daily relative strength (sector return - benchmark return).
    """
    # Align dates
    common_dates = sector_df.index.intersection(benchmark_df.index)
    
    sector_returns = sector_df.loc[common_dates, 'Returns']
    benchmark_returns = benchmark_df.loc[common_dates, 'Returns']
    
    relative_strength = sector_returns - benchmark_returns
    
    return relative_strength

print("✓ Relative strength function ready")

✓ Relative strength function ready


In [39]:
# =============================================================================
# ROTATION SIGNAL DETECTION
# =============================================================================

def find_rotation_signals(relative_strength, outperform_threshold, consecutive_days):
    """
    Find dates where sector outperformed for N consecutive days.
    """
    # Flag days where sector outperformed
    outperforming = (relative_strength > outperform_threshold).astype(int)
    
    # Find streaks
    signals = []
    streak = 0
    streak_start = None
    
    for date, value in outperforming.items():
        if value == 1:
            if streak == 0:
                streak_start = date
            streak += 1
            
            if streak == consecutive_days:
                signals.append({
                    'signal_date': date,
                    'streak_start': streak_start,
                    'streak_length': streak,
                    'avg_outperformance': relative_strength.loc[streak_start:date].mean()
                })
        else:
            streak = 0
            streak_start = None
    
    return pd.DataFrame(signals)

print("✓ Rotation signal detection ready")

✓ Rotation signal detection ready


In [40]:
# =============================================================================
# FORWARD RETURNS ANALYSIS
# =============================================================================

def analyze_post_rotation_returns(signals_df, sector_df, forward_periods):
    """
    After rotation signal, what did sector return?
    """
    results = []
    
    for _, signal in signals_df.iterrows():
        signal_date = signal['signal_date']
        
        if signal_date in sector_df.index:
            idx = sector_df.index.get_loc(signal_date)
            entry_price = sector_df.iloc[idx]['Adj Close']
            
            for period in forward_periods:
                if idx + period < len(sector_df):
                    exit_price = sector_df.iloc[idx + period]['Adj Close']
                    forward_return = (exit_price - entry_price) / entry_price
                    
                    results.append({
                        'signal_date': signal_date,
                        'avg_outperformance': signal['avg_outperformance'],
                        'forward_period': period,
                        'forward_return': forward_return,
                        'win': forward_return > 0
                    })
    
    return pd.DataFrame(results)

print("✓ Forward returns analysis ready")

✓ Forward returns analysis ready


In [41]:
# =============================================================================
# FULL ROTATION BACKTEST
# =============================================================================

def backtest_rotation_strategy(sector_etf, benchmark, 
                               outperform_thresholds=OUTPERFORM_THRESHOLDS,
                               consecutive_days_list=CONSECUTIVE_DAYS,
                               forward_periods=FORWARD_PERIODS):
    """
    Test all combinations.
    """
    print(f"\n📊 Loading data for {sector_etf} and {benchmark}...")
    
    # Load data
    sector_data = yf.download(sector_etf, start=START_DATE, end=END_DATE, progress=False)
    sector_data['Returns'] = sector_data['Adj Close'].pct_change()
    print(f"✓ {sector_etf}: {len(sector_data)} days")
    
    benchmark_data = yf.download(benchmark, start=START_DATE, end=END_DATE, progress=False)
    benchmark_data['Returns'] = benchmark_data['Adj Close'].pct_change()
    print(f"✓ {benchmark}: {len(benchmark_data)} days")
    
    # Calculate relative strength
    rel_strength = calculate_relative_strength(sector_data, benchmark_data)
    print(f"✓ Relative strength calculated")
    
    all_results = []
    
    print(f"\n🔍 Testing {len(outperform_thresholds)} thresholds × {len(consecutive_days_list)} streak lengths")
    print("="*60)
    
    for threshold in outperform_thresholds:
        for consec_days in consecutive_days_list:
            print(f"Testing: {threshold*100:.1f}% outperformance for {consec_days} days")
            
            # Find signals
            signals = find_rotation_signals(rel_strength, threshold, consec_days)
            print(f"  Found {len(signals)} rotation signals")
            
            if len(signals) == 0:
                continue
            
            # Analyze forward returns
            results = analyze_post_rotation_returns(signals, sector_data, forward_periods)
            
            if len(results) > 0:
                results['outperform_threshold'] = threshold
                results['consecutive_days'] = consec_days
                all_results.append(results)
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    return pd.DataFrame()

print("✓ Backtest engine ready")

✓ Backtest engine ready


In [42]:
# FIXED backtest function with multi-index handling
def backtest_rotation_strategy_v2(sector_etf, benchmark, 
                               outperform_thresholds=OUTPERFORM_THRESHOLDS,
                               consecutive_days_list=CONSECUTIVE_DAYS,
                               forward_periods=FORWARD_PERIODS):
    """
    Test all combinations - FIXED for yfinance multi-index.
    """
    print(f"\n📊 Loading data for {sector_etf} and {benchmark}...")
    
    # Load sector data
    sector_data = yf.download(sector_etf, start=START_DATE, end=END_DATE, progress=False)
    if isinstance(sector_data.columns, pd.MultiIndex):
        sector_data.columns = sector_data.columns.droplevel(1)
    if 'Adj Close' in sector_data.columns:
        sector_data['Returns'] = sector_data['Adj Close'].pct_change()
    elif 'Close' in sector_data.columns:
        sector_data['Returns'] = sector_data['Close'].pct_change()
        sector_data['Adj Close'] = sector_data['Close']
    print(f"✓ {sector_etf}: {len(sector_data)} days")
    
    # Load benchmark data
    benchmark_data = yf.download(benchmark, start=START_DATE, end=END_DATE, progress=False)
    if isinstance(benchmark_data.columns, pd.MultiIndex):
        benchmark_data.columns = benchmark_data.columns.droplevel(1)
    if 'Adj Close' in benchmark_data.columns:
        benchmark_data['Returns'] = benchmark_data['Adj Close'].pct_change()
    elif 'Close' in benchmark_data.columns:
        benchmark_data['Returns'] = benchmark_data['Close'].pct_change()
        benchmark_data['Adj Close'] = benchmark_data['Close']
    print(f"✓ {benchmark}: {len(benchmark_data)} days")
    
    # Calculate relative strength
    rel_strength = calculate_relative_strength(sector_data, benchmark_data)
    print(f"✓ Relative strength calculated")
    
    all_results = []
    
    print(f"\n🔍 Testing {len(outperform_thresholds)} thresholds × {len(consecutive_days_list)} streak lengths")
    print("="*60)
    
    for threshold in outperform_thresholds:
        for consec_days in consecutive_days_list:
            print(f"Testing: {threshold*100:.1f}% outperformance for {consec_days} days")
            
            # Find signals
            signals = find_rotation_signals(rel_strength, threshold, consec_days)
            print(f"  Found {len(signals)} rotation signals")
            
            if len(signals) == 0:
                continue
            
            # Analyze forward returns
            results = analyze_post_rotation_returns(signals, sector_data, forward_periods)
            
            if len(results) > 0:
                results['outperform_threshold'] = threshold
                results['consecutive_days'] = consec_days
                all_results.append(results)
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    return pd.DataFrame()

print("✓ Fixed backtest engine (v2)")

✓ Fixed backtest engine (v2)


In [43]:
# =============================================================================
# RUN BACKTEST with fixed function
# =============================================================================

print("\n🐺 STARTING SECTOR ROTATION BACKTEST")
print("="*60)

rotation_results = backtest_rotation_strategy_v2(SECTOR_ETF, BENCHMARK)

# Summarize
if len(rotation_results) > 0:
    summary = rotation_results.groupby(['outperform_threshold', 'consecutive_days', 'forward_period']).agg({
        'forward_return': ['count', 'mean', lambda x: (x > 0).mean()]
    }).reset_index()
    
    summary.columns = ['threshold', 'consec_days', 'forward_period', 'num_signals', 'avg_return', 'win_rate']
    summary['expected_value'] = summary['win_rate'] * summary['avg_return']
    summary = summary.sort_values('expected_value', ascending=False)
    
    print("\n" + "="*60)
    print("📊 TOP 20 SETUPS (sorted by expected value)")
    print("="*60)
    print(summary.head(20).to_string(index=False))
    
    # Show best
    best = summary.iloc[0]
    print(f"\n🎯 BEST SETUP:")
    print(f"   Outperform threshold: {best['threshold']*100:.1f}%")
    print(f"   Consecutive days: {int(best['consec_days'])}")
    print(f"   Forward period: {int(best['forward_period'])} days")
    print(f"   Number of signals: {int(best['num_signals'])}")
    print(f"   Win rate: {best['win_rate']*100:.1f}%")
    print(f"   Avg return: {best['avg_return']*100:.2f}%")
    print(f"   Expected value: {best['expected_value']*100:.2f}%")
else:
    print("\n❌ No results found")


🐺 STARTING SECTOR ROTATION BACKTEST

📊 Loading data for XBI and SPY...


✓ XBI: 506 days
✓ SPY: 506 days
✓ Relative strength calculated

🔍 Testing 4 thresholds × 4 streak lengths
Testing: 1.0% outperformance for 2 days
  Found 19 rotation signals
Testing: 1.0% outperformance for 3 days
  Found 4 rotation signals
Testing: 1.0% outperformance for 4 days
  Found 0 rotation signals
Testing: 1.0% outperformance for 5 days
  Found 0 rotation signals
Testing: 1.5% outperformance for 2 days
  Found 8 rotation signals
Testing: 1.5% outperformance for 3 days
  Found 0 rotation signals
Testing: 1.5% outperformance for 4 days
  Found 0 rotation signals
Testing: 1.5% outperformance for 5 days
  Found 0 rotation signals
Testing: 2.0% outperformance for 2 days
  Found 2 rotation signals
Testing: 2.0% outperformance for 3 days
  Found 0 rotation signals
Testing: 2.0% outperformance for 4 days
  Found 0 rotation signals
Testing: 2.0% outperformance for 5 days
  Found 0 rotation signals
Testing: 2.5% outperformance for 2 days
  Found 1 rotation signals
Testing: 2.5% outperfo

In [44]:
# =============================================================================
# EXECUTE - RUN THE BACKTEST
# =============================================================================

print("\n🐺 STARTING SECTOR ROTATION BACKTEST")
print("="*60)

rotation_results = backtest_rotation_strategy(SECTOR_ETF, BENCHMARK)

# Summarize
if len(rotation_results) > 0:
    rotation_summary = rotation_results.groupby(
        ['outperform_threshold', 'consecutive_days', 'forward_period']
    ).agg({
        'forward_return': ['mean', 'std', 'count'],
        'win': 'mean'
    }).round(4)
    
    rotation_summary.columns = ['avg_return', 'std', 'num_signals', 'win_rate']
    rotation_summary = rotation_summary.reset_index()
    rotation_summary['sharpe'] = rotation_summary['avg_return'] / rotation_summary['std']
    
    print("\n" + "="*60)
    print(f"📊 ROTATION STRATEGY RESULTS: {SECTOR_ETF} vs {BENCHMARK}")
    print("="*60)
    print(rotation_summary.sort_values('sharpe', ascending=False).head(20).to_string(index=False))
    
    # Best setup
    best = rotation_summary.sort_values('sharpe', ascending=False).iloc[0]
    print(f"\n🎯 BEST SETUP (by Sharpe):")
    print(f"   Threshold: {best['outperform_threshold']*100:.1f}%")
    print(f"   Consecutive days: {int(best['consecutive_days'])}")
    print(f"   Forward period: {int(best['forward_period'])} days")
    print(f"   Win rate: {best['win_rate']*100:.1f}%")
    print(f"   Avg return: {best['avg_return']*100:.2f}%")
    print(f"   Sharpe: {best['sharpe']:.2f}")
    print(f"   Number of signals: {int(best['num_signals'])}")
else:
    print("\n✗ No results found - try different parameters")


🐺 STARTING SECTOR ROTATION BACKTEST

📊 Loading data for XBI and SPY...


KeyError: 'Adj Close'

In [ ]:
# =============================================================================
# VISUALIZATION
# =============================================================================

if len(rotation_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'🐺 SECTOR ROTATION: {SECTOR_ETF} vs {BENCHMARK}', fontsize=16, fontweight='bold')
    
    # Win rate by forward period
    for consec_days in rotation_summary['consecutive_days'].unique():
        data = rotation_summary[rotation_summary['consecutive_days'] == consec_days]
        avg_by_period = data.groupby('forward_period')['win_rate'].mean()
        axes[0, 0].plot(avg_by_period.index, avg_by_period.values, 
                        marker='o', label=f'{int(consec_days)} days', linewidth=2)
    axes[0, 0].set_xlabel('Forward Period (days)', fontsize=12)
    axes[0, 0].set_ylabel('Win Rate', fontsize=12)
    axes[0, 0].set_title('Win Rate by Forward Period', fontweight='bold')
    axes[0, 0].legend(title='Consecutive Days')
    axes[0, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
    axes[0, 0].grid(alpha=0.3)
    
    # Average return by forward period
    for consec_days in rotation_summary['consecutive_days'].unique():
        data = rotation_summary[rotation_summary['consecutive_days'] == consec_days]
        avg_by_period = data.groupby('forward_period')['avg_return'].mean()
        axes[0, 1].plot(avg_by_period.index, avg_by_period.values*100, 
                        marker='o', label=f'{int(consec_days)} days', linewidth=2)
    axes[0, 1].set_xlabel('Forward Period (days)', fontsize=12)
    axes[0, 1].set_ylabel('Average Return (%)', fontsize=12)
    axes[0, 1].set_title('Average Return by Forward Period', fontweight='bold')
    axes[0, 1].legend(title='Consecutive Days')
    axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[0, 1].grid(alpha=0.3)
    
    # Sharpe by threshold
    for consec_days in rotation_summary['consecutive_days'].unique():
        data = rotation_summary[rotation_summary['consecutive_days'] == consec_days]
        avg_by_threshold = data.groupby('outperform_threshold')['sharpe'].mean()
        axes[1, 0].plot(avg_by_threshold.index*100, avg_by_threshold.values, 
                        marker='o', label=f'{int(consec_days)} days', linewidth=2)
    axes[1, 0].set_xlabel('Outperformance Threshold (%)', fontsize=12)
    axes[1, 0].set_ylabel('Sharpe Ratio', fontsize=12)
    axes[1, 0].set_title('Sharpe by Threshold', fontweight='bold')
    axes[1, 0].legend(title='Consecutive Days')
    axes[1, 0].grid(alpha=0.3)
    
    # Number of signals by setup
    signals_by_setup = rotation_summary.groupby(
        ['outperform_threshold', 'consecutive_days']
    )['num_signals'].mean().reset_index()
    
    for consec_days in signals_by_setup['consecutive_days'].unique():
        data = signals_by_setup[signals_by_setup['consecutive_days'] == consec_days]
        axes[1, 1].plot(data['outperform_threshold']*100, data['num_signals'], 
                        marker='o', label=f'{int(consec_days)} days', linewidth=2)
    axes[1, 1].set_xlabel('Outperformance Threshold (%)', fontsize=12)
    axes[1, 1].set_ylabel('Number of Signals', fontsize=12)
    axes[1, 1].set_title('Signal Frequency', fontweight='bold')
    axes[1, 1].legend(title='Consecutive Days')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize")

---

## 📊 INTERPRETATION

**What to look for:**
- **Win rate > 55%** = Rotation signal works
- **Sharpe > 1.0** = Risk-adjusted edge
- **10+ signals** = Enough occurrences to trust

**Key insights:**
- If 2-day streak has higher Sharpe than 5-day = early rotations are stronger
- If 10-day forward period best = rotations last 2 weeks
- If high threshold (2.5%) has fewer signals but better returns = quality over quantity

**Next steps:**
1. Test other sector ETFs (HACK, XLF, XLE, etc)
2. Compare to individual stock coordination (Notebook 4)
3. Combine with leader/laggard signals

🐺 **Money flows leave tracks. Follow them.**